<font color=blue>**NOME: Diego Batalha** </font>

# Introdução à Recuperação de Informações

## Lista 3

### Exercício 1:
A partir de um corpus à sua escolha, estime um modelo de assuntos baseado no Modelo LSI. Uma vez calculado o modelo, defina um conjunto de **documentos relevantes (${\cal R}$)** para um assunto, como os $n$ documentos que contiverem em sua representação LSI, os maiores coeficientes para o assunto escolhido. Construa uma consulta $q$, com as dez palavras mais importantes do assunto escolhido.

In [1]:
import os

import nltk
from nltk.corpus import machado
from nltk.stem.snowball import PortugueseStemmer
from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords

import string
from string import punctuation

from collections import defaultdict
from collections import OrderedDict

import numpy as np
from numpy.linalg import norm

import matplotlib.pylab as plt
import pandas as pd

import seaborn as sns

from pprint import pprint

## Definindo um tema para análise


In [2]:
textos = [machado.raw(id) for id in machado.fileids()]
swu = stopwords.words('portuguese') + list (string.punctuation)
stemmer = PortugueseStemmer()

In [3]:
textos_limpos = []
for texto in textos:
    tlimpo = [stemmer.stem(token.lower()) for token in WordPunctTokenizer().tokenize(texto) if token not in swu]
    textos_limpos.append(tlimpo)

In [13]:
dictionary = corpora.Dictionary(textos_limpos)
corpus = [dictionary.doc2bow(text) for text in textos_limpos]

In [32]:
mod_tfidf = models.TfidfModel(corpus)
corpus_tfidf = mod_tfidf[corpus]

In [ ]:
mod_lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=200)
corpus_lsi = mod_lsi[corpus_tfidf] 

In [16]:
topico = 5
mod_lsi.show_topic(topico,10)

[('cecíl', -0.56734219735658364),
 ('luís', 0.4283021670907971),
 ('alves', 0.20137512502482369),
 ('venânci', -0.13207811179098244),
 ('carlot', -0.1232977976792473),
 ('major', 0.11894631801782865),
 ('magalhã', -0.11637546023257263),
 ('albert', 0.11481660552969568),
 ('tibúrci', -0.10886793623477559),
 ('marcelin', 0.10287118355012102)]

In [17]:
lista = []
for i,texto in enumerate(corpus_lsi):
    lista.append((texto[topico][1],i))

R_relevantes = []
n_relevantes = 25
for i in sorted(lista,reverse = True)[:n_relevantes]:
    R_relevantes.append(i[1])

In [18]:
R_relevantes = set(R_relevantes)
print(R_relevantes)

{261, 265, 270, 15, 287, 416, 289, 288, 163, 38, 299, 45, 55, 441, 64, 456, 81, 82, 83, 93, 221, 481, 369, 244, 251}


In [19]:
t_relevantes = 10
q_consulta = []
for t in mod_lsi.show_topic(topico,10):
    q_consulta.append(t[0])

In [20]:
print(q_consulta)

['cecíl', 'luís', 'alves', 'venânci', 'carlot', 'major', 'magalhã', 'albert', 'tibúrci', 'marcelin']


In [7]:
print(dicionario)
print(corpus)
498*5886

NameError: name 'dicionario' is not defined

In [8]:
print(dicionario[0])
for doc in corpus[:3]:
    print(doc)

NameError: name 'dicionario' is not defined

### Exercício 2: 
Reutilizando os índices invertidos construídos em exercícios anteriores(Booleano, e TFIDF), calcule a precisão e revocação  com a consulta $q$ e o conjunto relevante ${\cal R}$ definidos no exercício anterior.

In [21]:
indice = defaultdict(lambda:set([]))
for tid,t in enumerate(textos_limpos):
    for term in t:
        indice[term].add(tid)

In [22]:
def busca_adv(search, indice):

    resultado = indice[search[0]]
    for token in search[1:]:
            resultado = resultado | indice[token]
    
    return resultado

In [23]:
resp_boo = busca_adv(q_consulta,indice)
resp_boo = set(resp_boo)
print(resp_boo)

{0, 1, 2, 4, 5, 11, 12, 13, 15, 16, 18, 21, 23, 24, 25, 26, 27, 28, 29, 33, 34, 36, 38, 39, 40, 43, 45, 48, 49, 50, 51, 52, 54, 55, 57, 59, 60, 61, 63, 64, 66, 67, 71, 76, 80, 81, 82, 83, 84, 92, 93, 95, 96, 105, 107, 110, 113, 120, 121, 122, 128, 139, 140, 151, 154, 158, 161, 162, 163, 165, 168, 171, 174, 175, 176, 179, 182, 183, 185, 187, 188, 190, 191, 192, 193, 205, 206, 207, 208, 210, 211, 217, 218, 219, 221, 222, 224, 227, 229, 230, 231, 232, 233, 234, 235, 239, 240, 242, 244, 245, 246, 249, 251, 254, 255, 256, 257, 258, 260, 261, 263, 265, 266, 267, 269, 270, 272, 273, 277, 282, 286, 287, 288, 289, 290, 298, 299, 301, 302, 311, 313, 316, 319, 326, 327, 328, 334, 345, 346, 357, 360, 364, 367, 368, 369, 371, 374, 377, 380, 381, 382, 385, 388, 389, 391, 393, 394, 396, 397, 398, 399, 411, 416, 423, 424, 425, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 441, 442, 443, 444, 446, 447, 448, 450, 451, 456, 463, 464, 465, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 48

In [24]:
T = nltk.TextCollection(textos_limpos)

tfidf_matrix = np.zeros((len(textos_limpos),len(q_consulta)))

for j, termo in enumerate(q_consulta):
    for i, texto in enumerate(textos_limpos):
        tfidf_matrix[i,j] = T.tf_idf(termo,texto)
        
Mtfidf_Norm = np.array([r/norm(r) if norm(r) !=0 else np.zeros(len(r)) for r in tfidf_matrix])

In [25]:
def ordem(q,MN):
    return [np.dot(q,r) for r in MN]

vetor_tfidf = np.array([T.tf_idf(w,q_consulta) for w in q_consulta])
vetor_tfidf /= norm(vetor_tfidf)
resp_tfidf = ordem(vetor_tfidf,Mtfidf_Norm)

vtfidf = filter(lambda x : x[0]!=0.0, zip(resp_tfidf,range(len(textos_limpos))))

temp_tfidf = sorted(vtfidf, reverse=True)

trunc = 0.3
resp_tfidf = []
for i in temp_tfidf:
    if i[0]>= trunc:
        resp_tfidf.append(i[1])

In [26]:
resp_tfidf = set(resp_tfidf)
print(resp_tfidf)

{0, 258, 2, 388, 389, 263, 391, 393, 11, 140, 397, 13, 399, 139, 15, 273, 272, 26, 158, 288, 161, 33, 34, 165, 38, 423, 168, 425, 299, 301, 430, 431, 48, 429, 52, 182, 183, 57, 442, 185, 443, 187, 441, 191, 447, 193, 66, 67, 326, 328, 206, 463, 208, 465, 82, 469, 470, 471, 217, 346, 219, 345, 93, 221, 95, 481, 482, 483, 487, 232, 364, 367, 239, 240, 371, 244, 374, 120, 122, 254}


In [27]:
def RevPre(parcial,geral):
    
    #parcial = 'documentos encontrados'
    #geral = 'documentos relevantes'
    
    rev = len(parcial & geral)/len(geral)
    pre = len(parcial & geral)/len(parcial)
    
    print('Revocação = ' + str(rev))
    print('Precisão = ' + str(pre))
    
    #return rev,pre

In [28]:
print('Resultados para a busca booleana')
RevPre(resp_boo,R_relevantes)
print('\n')
print('Resultados para a busca por tf_idf')
RevPre(resp_tfidf,R_relevantes)

Resultados para a busca booleana
Revocação = 1.0
Precisão = 0.11363636363636363


Resultados para a busca por tf_idf
Revocação = 0.4
Precisão = 0.125


### Exercício 3: 
Usando as definições de probabilidade de relevância apresentadas no capítulo 11 do Livro, construa uma função de recuperação probabilística usando o *log da razão de Odds* como **RSV** (retrieval status value). Calcule revocação e precisão para consulta $q$ e conjunto relevante ${\cal R}$. Compare a probabilidade $p_t=P(x_t=1|R=1,q)$, com a o rankeamento de importância das palavras que compõem o assunto escolhido.

In [29]:
def prob_busca(query, indice, relevantes, N_docs = 246):
    
    #query = consulta a ser realizada
    #indice = indice invertido
    #relevantes = conjunto de documentos atualmente considerados relevantes
    #N_docs = número de documentos presente no corpus
    
    query_tok = query #[stemmer.stem(token.lower())  for token in WordPunctTokenizer().tokenize(query) if token not in swu]
    
    ct = []
    S = len(relevantes)
    N = N_docs
    
    for termo in query_tok:
        s = len(indice[termo] & relevantes)
        df = len(indice[termo])
        
        up = s/(S - s +0.5)
        down =(df - s)/(N - df - S + s -0.5)
        
        ct.append(np.log(up/down))
    
    RSVs = []
    for doc in range(N):
        rsv = 0
        for pos,termo in enumerate(query_tok):
            if doc in indice[termo]:
                rsv += ct[pos]
        RSVs.append((rsv,doc))
        
    resultado = sorted(RSVs,reverse=True)
    
    return resultado

In [30]:
temp_prob = prob_busca(q_consulta,indice,R_relevantes)
print(temp_prob[:10])

[(4.6817989334694596, 57), (4.3926449489384423, 82), (4.0787876747445351, 219), (4.0787876747445351, 13), (4.0584755342905527, 93), (3.4385386159004221, 207), (3.4385386159004221, 1), (3.4182264754464398, 221), (3.4182264754464398, 15), (3.0840570607985502, 244)]


In [31]:
trunc = 1
resp_prob = []
for i in temp_prob:
    if i[0] >= trunc:
        resp_prob.append(i[1])
        
resp_prob = set(resp_prob)
print(resp_prob)

{0, 1, 4, 12, 13, 15, 21, 23, 27, 28, 158, 34, 163, 36, 38, 39, 45, 176, 49, 51, 182, 57, 59, 188, 61, 192, 64, 205, 206, 207, 81, 82, 210, 83, 218, 219, 92, 221, 93, 96, 227, 229, 233, 234, 240, 113, 242, 244, 245, 122}


In [33]:
RevPre(resp_prob,R_relevantes)

Revocação = 0.44
Precisão = 0.22


In [34]:
p_termo = []
S = len(R_relevantes)
for termo in q_consulta:
    s = len(indice[termo] & R_relevantes)
    p_termo.append((termo,s/S))

In [35]:
print(p_termo)
print('\n')
print(mod_lsi.show_topic(topico,10))

[('cecíl', 0.0), ('luís', 0.92), ('alves', 0.16), ('venânci', 0.0), ('carlot', 0.16), ('major', 0.28), ('magalhã', 0.0), ('albert', 0.16), ('tibúrci', 0.08), ('marcelin', 0.16)]


[('cecíl', -0.56734219735658364), ('luís', 0.4283021670907971), ('alves', 0.20137512502482369), ('venânci', -0.13207811179098244), ('carlot', -0.1232977976792473), ('major', 0.11894631801782865), ('magalhã', -0.11637546023257263), ('albert', 0.11481660552969568), ('tibúrci', -0.10886793623477559), ('marcelin', 0.10287118355012102)]


### Exercício 4:
Repita o exercício 3 agora usando o modelo **Okapi BM25** para o rankeamento. Calcule precisão e revocação.

In [36]:
def okapi_busca(query, indice, corpo):
    
    #query = consulta a ser realizada
    #indice = indice invertido
    #corpo = corpus
    
    query_tok = query #[stemmer.stem(token.lower())  for token in WordPunctTokenizer().tokenize(query) if token not in swu]
    
    N = len(corpo)
    
    L = []
    for texto in corpo:
        L.append(len(texto))
        
    L_ave = sum(L)/N
    k1 = 1.2
    b = 0.75
    
    RSVs = []
    for doc in range(N):
        rsv = 0
        
        for termo in query_tok:
            if doc in indice[termo]:
                tf = 1 #pode ser alterado para melhor representar a frequencia
            df = len(indice[termo])
        rsv += np.log(N/df)*((k1 + 1)*tf)/(k1*(1 - b + (b*(L[doc]/L_ave))) + tf)
        
        RSVs.append((rsv,doc))
        
    resultado = sorted(RSVs,reverse=True)
    
    return resultado

In [37]:
temp_okapi = okapi_busca(q_consulta,indice, textos_limpos)
print(temp_okapi[:10])

[(6.2393542810351423, 405), (6.2393542810351423, 199), (6.2027710326330876, 457), (6.2027710326330876, 417), (6.2007879168888991, 410), (6.2007879168888991, 204), (6.1522693329462017, 403), (6.1522693329462017, 197), (6.1490184201635358, 376), (6.1490184201635358, 170)]


In [38]:
trunc_okapi = 1
resp_okapi = []
for i in temp_okapi:
    if i[0] >= trunc_okapi:
        resp_okapi.append(i[1])
        
resp_okapi = set(resp_okapi)
print(resp_prob)

{0, 1, 4, 12, 13, 15, 21, 23, 27, 28, 158, 34, 163, 36, 38, 39, 45, 176, 49, 51, 182, 57, 59, 188, 61, 192, 64, 205, 206, 207, 81, 82, 210, 83, 218, 219, 92, 221, 93, 96, 227, 229, 233, 234, 240, 113, 242, 244, 245, 122}


In [39]:
RevPre(resp_okapi,R_relevantes)

Revocação = 1.0
Precisão = 0.05122950819672131
